In [25]:
from enum import Enum
from experta import *

In [26]:
class Problems(Enum):
    HOT_WATER_OR_STEAM = "Hot water or steam generation impossible"
    MILK_FROTH_WEAK = "Milk froth is too weak or no milk froth"
    COFFEE_DROPS = "Coffee flows only drop by drop"
    NO_CREMA = "No crema on the coffee"
    LOUD_GRINDER_NOISE = "Loud grinder noise"
    ERROR_MESSAGE = "Error message 8 on display"

class Checks(Enum):
    SYSTEM_CALCIFICATION = "Is the system possibly calcified?"
    SPUMATORE_CLOGGED = "Is the spumatore clogged?"
    INAPPROPRIATE_MILK = "Are you using inappropriate milk?"
    GRIND_SIZE_FINE = "Is the grind size too fine?"
    COFFEE_TOO_FINE = "Is the coffee too finely ground?"
    TOO_MUCH_COFFEE = "Is there too much coffee?"
    BREWING_UNIT_CLOGGED = "Is the brewing unit clogged?"
    GRIND_SIZE_NOT_ADJUSTED = "Is the grind size not adjusted to the coffee beans?"
    FOREIGN_OBJECTS = "Are there any foreign objects in the grinder?"
    BREWING_UNIT_POSITION_INCORRECT = "Is the brewing unit position incorrect?"

class Causes(Enum):
    SYSTEM_CALCIFICATION = "system calcification"
    SPUMATORE_CLOGGED = "spumatore clogged"
    INAPPROPRIATE_MILK = "inappropriate milk"
    GRIND_SIZE_TOO_FINE = "grind size too fine"
    COFFEE_TOO_FINE = "coffee too finely ground"
    TOO_MUCH_COFFEE = "too much coffee"
    BREWING_UNIT_CLOGGED = "brewing unit clogged"
    GRIND_SIZE_NOT_ADJUSTED = "grind size not adjusted to coffee beans"
    FOREIGN_OBJECTS = "foreign objects in grinder"
    BREWING_UNIT_POSITION_INCORRECT = "brewing unit position incorrect"
    UNKNOWN_ISSUE = "unknown issue"

class Actions(Enum):
    DESCALE_SYSTEM = "Descale the system with a high dose of descaling agent"
    CLEAN_SPUMATORE = "Thoroughly clean the spumatore, disassemble it completely"
    USE_COLD_MILK = "Use cold milk"
    SET_COARSER_GRIND_SIZE = "Set a coarser grind size"
    USE_COARSER_GROUND_COFFEE = "Use coarser ground coffee"
    USE_LESS_GROUND_COFFEE = "Use less ground coffee"
    CLEAN_BREWING_UNIT = "Remove and clean the brewing unit"
    OPTIMIZE_GRIND_SIZE = "Optimize the grind size"
    REMOVE_FOREIGN_OBJECTS = "Remove the foreign objects or contact service"
    RESET_BREWING_UNIT = "Turn off the device and remove the plug. Reinsert the plug and turn on the device. When ready, remove and clean the brewing unit"
    CONTACT_SERVICE = "Seems like a bigger issue - please contact professional service"

In [27]:
class Check(Fact):
    """Represents a check result for a specific problem"""
    pass

class Cause(Fact):
    """Represents a cause of a problem"""
    pass

class Action(Fact):
    """Represents an action to be taken to solve a problem"""
    pass

In [28]:
class CoffeeMachineExpert(KnowledgeEngine):
    @DefFacts()
    def init(self):
        for check in Checks:
            yield Check(description=check.value, response=None)

    @Rule(Check(description=MATCH.description, response=None))
    def ask_check(self, description):
        response = input(f"{description} (yes/no): ").strip().lower()
        self.declare(Check(description=description, response=response))

    # Define rules for each problem leading to a cause
    @Rule(AND(Fact(problem=Problems.HOT_WATER_OR_STEAM.value),
              Check(description=Checks.SYSTEM_CALCIFICATION.value, response='yes')))
    def diagnose_system_calcification(self):
        self.declare(Cause(description=Causes.SYSTEM_CALCIFICATION.value))
        self.declare(Action(action=Actions.DESCALE_SYSTEM.value))

    @Rule(AND(Fact(problem=Problems.MILK_FROTH_WEAK.value),
              Check(description=Checks.SPUMATORE_CLOGGED.value, response='yes')))
    def diagnose_spumatore_clogged(self):
        self.declare(Cause(description=Causes.SPUMATORE_CLOGGED.value))
        self.declare(Action(action=Actions.CLEAN_SPUMATORE.value))

    @Rule(AND(Fact(problem=Problems.MILK_FROTH_WEAK.value),
              Check(description=Checks.SPUMATORE_CLOGGED.value, response='no'),
              Check(description=Checks.INAPPROPRIATE_MILK.value, response='yes')))
    def diagnose_inappropriate_milk(self):
        self.declare(Cause(description=Causes.INAPPROPRIATE_MILK.value))
        self.declare(Action(action=Actions.USE_COLD_MILK.value))

    @Rule(AND(Fact(problem=Problems.COFFEE_DROPS.value),
              Check(description=Checks.GRIND_SIZE_FINE.value, response='yes')))
    def diagnose_grind_size_fine(self):
        self.declare(Cause(description=Causes.GRIND_SIZE_TOO_FINE.value))
        self.declare(Action(action=Actions.SET_COARSER_GRIND_SIZE.value))

    @Rule(AND(Fact(problem=Problems.COFFEE_DROPS.value),
              Check(description=Checks.GRIND_SIZE_FINE.value, response='no'),
              Check(description=Checks.COFFEE_TOO_FINE.value, response='yes')))
    def diagnose_coffee_too_fine(self):
        self.declare(Cause(description=Causes.COFFEE_TOO_FINE.value))
        self.declare(Action(action=Actions.USE_COARSER_GROUND_COFFEE.value))

    @Rule(AND(Fact(problem=Problems.COFFEE_DROPS.value),
              Check(description=Checks.GRIND_SIZE_FINE.value, response='no'),
              Check(description=Checks.COFFEE_TOO_FINE.value, response='no'),
              Check(description=Checks.TOO_MUCH_COFFEE.value, response='yes')))
    def diagnose_too_much_coffee(self):
        self.declare(Cause(description=Causes.TOO_MUCH_COFFEE.value))
        self.declare(Action(action=Actions.USE_LESS_GROUND_COFFEE.value))

    @Rule(AND(Fact(problem=Problems.NO_CREMA.value),
              Check(description=Checks.BREWING_UNIT_CLOGGED.value, response='yes')))
    def diagnose_brewing_unit_clogged(self):
        self.declare(Cause(description=Causes.BREWING_UNIT_CLOGGED.value))
        self.declare(Action(action=Actions.CLEAN_BREWING_UNIT.value))

    @Rule(AND(Fact(problem=Problems.LOUD_GRINDER_NOISE.value),
              Check(description=Checks.GRIND_SIZE_NOT_ADJUSTED.value, response='yes')))
    def diagnose_grind_size_not_adjusted(self):
        self.declare(Cause(description=Causes.GRIND_SIZE_NOT_ADJUSTED.value))
        self.declare(Action(action=Actions.OPTIMIZE_GRIND_SIZE.value))

    @Rule(AND(Fact(problem=Problems.LOUD_GRINDER_NOISE.value),
              Check(description=Checks.GRIND_SIZE_NOT_ADJUSTED.value, response='no'),
              Check(description=Checks.FOREIGN_OBJECTS.value, response='yes')))
    def diagnose_foreign_objects_in_grinder(self):
        self.declare(Cause(description=Causes.FOREIGN_OBJECTS.value))
        self.declare(Action(action=Actions.REMOVE_FOREIGN_OBJECTS.value))

    @Rule(AND(Fact(problem=Problems.ERROR_MESSAGE.value),
              Check(description=Checks.BREWING_UNIT_POSITION_INCORRECT.value, response='yes')))
    def diagnose_brewing_unit_position_incorrect(self):
        self.declare(Cause(description=Causes.BREWING_UNIT_POSITION_INCORRECT.value))
        self.declare(Action(action=Actions.RESET_BREWING_UNIT.value))

    @Rule(AND(
        Check(description=MATCH.description, response='no')
        for description in [Checks.SYSTEM_CALCIFICATION.value, Checks.SPUMATORE_CLOGGED.value, Checks.COFFEE_TOO_FINE.value, Checks.TOO_MUCH_COFFEE.value, Checks.FOREIGN_OBJECTS.value, Checks.BREWING_UNIT_POSITION_INCORRECT.value]
    ))
    def diagnose_unknown_issue(self):
        self.declare(Cause(description=Causes.UNKNOWN_ISSUE.value))
        self.declare(Action(action=Actions.CONTACT_SERVICE.value))

    # Solution implementations
    @Rule(Action(action=MATCH.action))
    def implement_action(self, action):
        print(f"Solution: {action}")

    # Print cause
    @Rule(Cause(description=MATCH.description))
    def print_cause(self, description):
        print(f"Detected Cause: {description}")

In [29]:
def ask_user(engine):
    problems = {
        1: Problems.HOT_WATER_OR_STEAM.value,
        2: Problems.MILK_FROTH_WEAK.value,
        3: Problems.COFFEE_DROPS.value,
        4: Problems.NO_CREMA.value,
        5: Problems.LOUD_GRINDER_NOISE.value,
        6: Problems.ERROR_MESSAGE.value
    }

    print("Choose the problem:")
    for key, value in problems.items():
        print(f"{key}. {value}")
    problem_choice = int(input("Enter the number of the problem: "))
    problem_description = problems[problem_choice]
    engine.declare(Fact(problem=problem_description))
    engine.run()

In [30]:
engine = CoffeeMachineExpert()
engine.reset()

# Ask user about the problem and cause
ask_user(engine)

ZeroDivisionError: division by zero

In [31]:
from enum import Enum
from experta import *

class Problems(Enum):
    HOT_WATER_OR_STEAM = "Hot water or steam generation impossible"
    MILK_FROTH_WEAK = "Milk froth is too weak or no milk froth"
    COFFEE_DROPS = "Coffee flows only drop by drop"
    NO_CREMA = "No crema on the coffee"
    LOUD_GRINDER_NOISE = "Loud grinder noise"
    ERROR_MESSAGE = "Error message 8 on display"

class Checks(Enum):
    SYSTEM_CALCIFICATION = "Is the system possibly calcified?"
    SPUMATORE_CLOGGED = "Is the spumatore clogged?"
    INAPPROPRIATE_MILK = "Are you using inappropriate milk?"
    GRIND_SIZE_FINE = "Is the grind size too fine?"
    COFFEE_TOO_FINE = "Is the coffee too finely ground?"
    TOO_MUCH_COFFEE = "Is there too much coffee?"
    BREWING_UNIT_CLOGGED = "Is the brewing unit clogged?"
    GRIND_SIZE_NOT_ADJUSTED = "Is the grind size not adjusted to the coffee beans?"
    FOREIGN_OBJECTS = "Are there any foreign objects in the grinder?"
    BREWING_UNIT_POSITION_INCORRECT = "Is the brewing unit position incorrect?"

class Causes(Enum):
    SYSTEM_CALCIFICATION = "system calcification"
    SPUMATORE_CLOGGED = "spumatore clogged"
    INAPPROPRIATE_MILK = "inappropriate milk"
    GRIND_SIZE_TOO_FINE = "grind size too fine"
    COFFEE_TOO_FINE = "coffee too finely ground"
    TOO_MUCH_COFFEE = "too much coffee"
    BREWING_UNIT_CLOGGED = "brewing unit clogged"
    GRIND_SIZE_NOT_ADJUSTED = "grind size not adjusted to coffee beans"
    FOREIGN_OBJECTS = "foreign objects in grinder"
    BREWING_UNIT_POSITION_INCORRECT = "brewing unit position incorrect"
    UNKNOWN_ISSUE = "unknown issue"

class Actions(Enum):
    DESCALE_SYSTEM = "Descale the system with a high dose of descaling agent"
    CLEAN_SPUMATORE = "Thoroughly clean the spumatore, disassemble it completely"
    USE_COLD_MILK = "Use cold milk"
    SET_COARSER_GRIND_SIZE = "Set a coarser grind size"
    USE_COARSER_GROUND_COFFEE = "Use coarser ground coffee"
    USE_LESS_GROUND_COFFEE = "Use less ground coffee"
    CLEAN_BREWING_UNIT = "Remove and clean the brewing unit"
    OPTIMIZE_GRIND_SIZE = "Optimize the grind size"
    REMOVE_FOREIGN_OBJECTS = "Remove the foreign objects or contact service"
    RESET_BREWING_UNIT = "Turn off the device and remove the plug. Reinsert the plug and turn on the device. When ready, remove and clean the brewing unit"
    CONTACT_SERVICE = "Seems like a bigger issue - please contact professional service"

class Check(Fact):
    """Represents a check result for a specific problem"""
    pass

class Cause(Fact):
    """Represents a cause of a problem"""
    pass

class Action(Fact):
    """Represents an action to be taken to solve a problem"""
    pass

class CoffeeMachineExpert(KnowledgeEngine):
    @DefFacts()
    def init(self):
        for check in Checks:
            yield Check(description=check.value, response=None)

    @Rule(Check(description=MATCH.description, response=None))
    def ask_check(self, description):
        response = input(f"{description} (yes/no): ").strip().lower()
        self.declare(Check(description=description, response=response))

    # Define rules for each problem leading to a cause
    @Rule(AND(Fact(problem=Problems.HOT_WATER_OR_STEAM.value),
              Check(description=Checks.SYSTEM_CALCIFICATION.value, response='yes')))
    def diagnose_system_calcification(self):
        self.declare(Cause(description=Causes.SYSTEM_CALCIFICATION.value))
        self.declare(Action(action=Actions.DESCALE_SYSTEM.value))

    @Rule(AND(Fact(problem=Problems.MILK_FROTH_WEAK.value),
              Check(description=Checks.SPUMATORE_CLOGGED.value, response='yes')))
    def diagnose_spumatore_clogged(self):
        self.declare(Cause(description=Causes.SPUMATORE_CLOGGED.value))
        self.declare(Action(action=Actions.CLEAN_SPUMATORE.value))

    @Rule(AND(Fact(problem=Problems.MILK_FROTH_WEAK.value),
              Check(description=Checks.SPUMATORE_CLOGGED.value, response='no'),
              Check(description=Checks.INAPPROPRIATE_MILK.value, response='yes')))
    def diagnose_inappropriate_milk(self):
        self.declare(Cause(description=Causes.INAPPROPRIATE_MILK.value))
        self.declare(Action(action=Actions.USE_COLD_MILK.value))

    @Rule(AND(Fact(problem=Problems.COFFEE_DROPS.value),
              Check(description=Checks.GRIND_SIZE_FINE.value, response='yes')))
    def diagnose_grind_size_fine(self):
        self.declare(Cause(description=Causes.GRIND_SIZE_TOO_FINE.value))
        self.declare(Action(action=Actions.SET_COARSER_GRIND_SIZE.value))

    @Rule(AND(Fact(problem=Problems.COFFEE_DROPS.value),
              Check(description=Checks.GRIND_SIZE_FINE.value, response='no'),
              Check(description=Checks.COFFEE_TOO_FINE.value, response='yes')))
    def diagnose_coffee_too_fine(self):
        self.declare(Cause(description=Causes.COFFEE_TOO_FINE.value))
        self.declare(Action(action=Actions.USE_COARSER_GROUND_COFFEE.value))

    @Rule(AND(Fact(problem=Problems.COFFEE_DROPS.value),
              Check(description=Checks.GRIND_SIZE_FINE.value, response='no'),
              Check(description=Checks.COFFEE_TOO_FINE.value, response='no'),
              Check(description=Checks.TOO_MUCH_COFFEE.value, response='yes')))
    def diagnose_too_much_coffee(self):
        self.declare(Cause(description=Causes.TOO_MUCH_COFFEE.value))
        self.declare(Action(action=Actions.USE_LESS_GROUND_COFFEE.value))

    @Rule(AND(Fact(problem=Problems.NO_CREMA.value),
              Check(description=Checks.BREWING_UNIT_CLOGGED.value, response='yes')))
    def diagnose_brewing_unit_clogged(self):
        self.declare(Cause(description=Causes.BREWING_UNIT_CLOGGED.value))
        self.declare(Action(action=Actions.CLEAN_BREWING_UNIT.value))

    @Rule(AND(Fact(problem=Problems.LOUD_GRINDER_NOISE.value),
              Check(description=Checks.GRIND_SIZE_NOT_ADJUSTED.value, response='yes')))
    def diagnose_grind_size_not_adjusted(self):
        self.declare(Cause(description=Causes.GRIND_SIZE_NOT_ADJUSTED.value))
        self.declare(Action(action=Actions.OPTIMIZE_GRIND_SIZE.value))

    @Rule(AND(Fact(problem=Problems.LOUD_GRINDER_NOISE.value),
              Check(description=Checks.GRIND_SIZE_NOT_ADJUSTED.value, response='no'),
              Check(description=Checks.FOREIGN_OBJECTS.value, response='yes')))
    def diagnose_foreign_objects_in_grinder(self):
        self.declare(Cause(description=Causes.FOREIGN_OBJECTS.value))
        self.declare(Action(action=Actions.REMOVE_FOREIGN_OBJECTS.value))

    @Rule(AND(Fact(problem=Problems.ERROR_MESSAGE.value),
              Check(description=Checks.BREWING_UNIT_POSITION_INCORRECT.value, response='yes')))
    def diagnose_brewing_unit_position_incorrect(self):
        self.declare(Cause(description=Causes.BREWING_UNIT_POSITION_INCORRECT.value))
        self.declare(Action(action=Actions.RESET_BREWING_UNIT.value))

    @Rule(AND(
        Check(description=Checks.SYSTEM_CALCIFICATION.value, response='no'),
        Check(description=Checks.SPUMATORE_CLOGGED.value, response='no'),
        Check(description=Checks.COFFEE_TOO_FINE.value, response='no'),
        Check(description=Checks.TOO_MUCH_COFFEE.value, response='no'),
        Check(description=Checks.FOREIGN_OBJECTS.value, response='no'),
        Check(description=Checks.BREWING_UNIT_POSITION_INCORRECT.value, response='no')
    ))
    def diagnose_unknown_issue(self):
        self.declare(Cause(description=Causes.UNKNOWN_ISSUE.value))
        self.declare(Action(action=Actions.CONTACT_SERVICE.value))

    # Solution implementations
    @Rule(Action(action=MATCH.action))
    def implement_action(self, action):
        print(f"Solution: {action}")

    # Print cause
    @Rule(Cause(description=MATCH.description))
    def print_cause(self, description):
        print(f"Detected Cause: {description}")

def ask_user(engine):
    problems = {
        1: Problems.HOT_WATER_OR_STEAM.value,
        2: Problems.MILK_FROTH_WEAK.value,
        3: Problems.COFFEE_DROPS.value,
        4: Problems.NO_CREMA.value,
        5: Problems.LOUD_GRINDER_NOISE.value,
        6: Problems.ERROR_MESSAGE.value
    }

    print("Choose the problem:")
    for key, value in problems.items():
        print(f"{key}. {value}")
    problem_choice = int(input("Enter the number of the problem: "))
    problem_description = problems[problem_choice]
    engine.declare(Fact(problem=problem_description))
    engine.run()

In [32]:
engine = CoffeeMachineExpert()
engine.reset()

# Ask user about the problem and cause
ask_user(engine)

Choose the problem:
1. Hot water or steam generation impossible
2. Milk froth is too weak or no milk froth
3. Coffee flows only drop by drop
4. No crema on the coffee
5. Loud grinder noise
6. Error message 8 on display
